# Demonstration of the CNN used for time series classification

## Data preparation

In order to feed data to our model, we need to turn it into a keras sequence. This way it is fed to the network as batches of 1 sample each time, and there is no issue with the fact the time series have a different size. The whole preparation is done in the data_preparation function. The data are also shuffled.

In [1]:
from cnn import Turtlebot_CNN,Turtlebot_LSTM,data_preparation
from read_data import make_dataset,read_bag
import tensorflow as tf
import matplotlib.pyplot as plt
import math
from inception import Inception


ds,l=data_preparation(path='/home/rwu-lh/FullData/')


ng_goals_2021-06-30-11-41-29 already exists. Not creating.
[INFO]  Data folder /home/rwu-lh/FullData/OdomNoise/42/auto_sending_goals_2021-06-30-11-21-01 already exists. Not creating.
[INFO]  Data folder /home/rwu-lh/FullData/OdomNoise/34/auto_sending_goals_2021-06-30-11-02-56 already exists. Not creating.
[INFO]  Data folder /home/rwu-lh/FullData/OdomNoise/17/auto_sending_goals_2021-06-29-11-53-49 already exists. Not creating.
[INFO]  Data folder /home/rwu-lh/FullData/OdomNoise/9/auto_sending_goals_2021-06-29-11-36-54 already exists. Not creating.
[INFO]  Data folder /home/rwu-lh/FullData/OdomNoise/35/auto_sending_goals_2021-06-30-11-05-12 already exists. Not creating.
[INFO]  Data folder /home/rwu-lh/FullData/OdomNoise/26/auto_sending_goals_2021-06-29-12-14-09 already exists. Not creating.
[INFO]  Data folder /home/rwu-lh/FullData/OdomNoise/21/auto_sending_goals_2021-06-29-12-02-51 already exists. Not creating.
[INFO]  Data folder /home/rwu-lh/FullData/OdomNoise/50/auto_sending_goals_

In [2]:
ds=ds.shuffle(l)
prop_train_val=0.8
l_train_val=math.ceil(prop_train_val*l)
train_val=ds.take(l_train_val)
test=ds.skip(l_train_val)
prop_train=0.8
l_train=math.floor(prop_train*l_train_val)
train=train_val.take(l_train)
validation=train_val.skip(l_train)
print(l_train)

115


In [3]:
train=train.batch(10)
test=test.batch(1)
validation=validation.batch(1)

In [5]:
inc=Inception()
cb_list_inc=[tf.keras.callbacks.ModelCheckpoint('inc_model/checkpoint_inc.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)]#tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=50),
inc.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[tf.keras.metrics.CategoricalAccuracy()])
hist_inc=inc.fit(train,epochs=500,callbacks=cb_list_inc,verbose=1,validation_data=validation)

Epoch 1/500
12/12 [==============================] - 23s 2s/step - loss: 1.2053 - categorical_accuracy: 0.4609 - val_loss: 2.1067 - val_categorical_accuracy: 0.2759
Epoch 2/500
12/12 [==============================] - 17s 1s/step - loss: 1.0337 - categorical_accuracy: 0.5478 - val_loss: 1.8355 - val_categorical_accuracy: 0.4138
Epoch 3/500
12/12 [==============================] - 17s 1s/step - loss: 0.8736 - categorical_accuracy: 0.6348 - val_loss: 3.4675 - val_categorical_accuracy: 0.3448
Epoch 4/500
12/12 [==============================] - 17s 1s/step - loss: 0.8629 - categorical_accuracy: 0.6435 - val_loss: 2.7632 - val_categorical_accuracy: 0.3448
Epoch 5/500
12/12 [==============================] - 17s 1s/step - loss: 0.8819 - categorical_accuracy: 0.6174 - val_loss: 3.6906 - val_categorical_accuracy: 0.3793
Epoch 6/500
 7/12 [================>.............] - ETA: 7s - loss: 0.7243 - categorical_accuracy: 0.6857

KeyboardInterrupt: 

In [ ]:
inc.load_weights('inc_model/checkpoint_inc.h5')
plt.figure()
plt.plot(hist_inc.history['loss'],label='Training Loss')
plt.plot(hist_inc.history['val_loss'],label='Validation Loss')
plt.plot(hist_inc.history['categorical_accuracy'],label='Training Accuracy')
plt.plot(hist_inc.history['val_categorical_accuracy'],label='Validation Accuracy')
plt.legend()
plt.show()
inc.evaluate(test)

## Training the Model

We can now train the model using the fit method.

In [6]:

lstm=Turtlebot_LSTM()
cb_list_lstm=[tf.keras.callbacks.ModelCheckpoint('lstm_model/checkpoint_lstm_new.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)]#tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=50),
lstm.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[tf.keras.metrics.CategoricalAccuracy()])
hist_lstm=lstm.fit(train,epochs=500,callbacks=cb_list_lstm,verbose=1,validation_data=validation)

12/12 [==============================] - 14s 1s/step - loss: 1.0940 - categorical_accuracy: 0.3913 - val_loss: 1.0682 - val_categorical_accuracy: 0.4828


In [ ]:
lstm.load_weights('lstm_model/checkpoint_lstm.h5')
plt.figure()
plt.plot(hist_lstm.history['loss'],label='Training Loss')
plt.plot(hist_lstm.history['val_loss'],label='Validation Loss')
plt.plot(hist_lstm.history['categorical_accuracy'],label='Training Accuracy')
plt.plot(hist_lstm.history['val_categorical_accuracy'],label='Validation Accuracy')
plt.legend()
plt.show()
lstm.evaluate(test)

In [4]:
cnn=Turtlebot_CNN()
cb_list_cnn=[tf.keras.callbacks.ModelCheckpoint('cnn_model/checkpoint_cnn_new.h5',monitor='val_loss',save_best_only=True,save_weights_only=True)]#,tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=50)]
cnn.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[tf.keras.metrics.CategoricalAccuracy()])
hist_cnn=cnn.fit(train,epochs=1,callbacks=cb_list_cnn,verbose=1,validation_data=validation)
cnn.summary()

(None, 32)
(None, 32)
(None, 32)
(None, 32)
112/115 [============================>.] - ETA: 0s - loss: 1.1523 - categorical_accuracy: 0.4375(None, 32)
(None, 32)
115/115 [==============================] - 2s 14ms/step - loss: 1.1479 - categorical_accuracy: 0.4435 - val_loss: 0.9142 - val_categorical_accuracy: 0.6552
Model: "turtlebot_cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              multiple                  8384      
_________________________________________________________________
conv1d_1 (Conv1D)            multiple                  20512     
_________________________________________________________________
conv1d_2 (Conv1D)            multiple                  460928    
_________________________________________________________________
conv1d_3 (Conv1D)            multiple                  81984     
_________________________________________________________________

In [ ]:
cnn.load_weights('cnn_model/checkpoint_cnn_new.h5')

plt.figure()
plt.plot(hist_cnn.history['loss'],label='Training Loss')
plt.plot(hist_cnn.history['val_loss'],label='Validation Loss')
plt.plot(hist_cnn.history['categorical_accuracy'],label='Training Accuracy')
plt.plot(hist_cnn.history['val_categorical_accuracy'],label='Validation Accuracy')
plt.legend()
plt.show()
cnn.evaluate(test)

## Testing the model

Here is a Visualization of what the CNN does 

![Visualization of the CNN](img/Schema_CNN.png)

In [ ]:
import numpy as np
def results(model,test):
    res=model.predict(test)
    c=0
    for i,pred in enumerate(res):
        y=test[i][1][0]
        if np.argmax(y)==np.argmax(pred):
            c+=1
    return(c/len(res))
            

In [ ]:
lstm.load_weights('lstm_model/checkpoint_lstm.h5')
cnn.load_weights('cnn_model/checkpoint_cnn.h5')
print('LSTM :',results(lstm,test))
print('CNN :',results(cnn,test))

In [ ]:
model.predict(test)

In [ ]:
train[0]

In [ ]:
tf.data.Dataset.from_generator(train)

In [ ]:
odoms,scans,labels=make_dataset('/media/louis/TOSHIBA EXT/FullData')

In [ ]:
tf.data.Dataset.from_generator(lambda :odoms,output_shapes=[None,13],output_types=tf.float32)

In [ ]:
np.shape(odoms[1])

In [ ]:
lstm=Turtlebot_CNN(type='lstm')
lstm.compile(optimizer='adam',loss='categorical_crossentropy')
lstm.load_weights('lstm_model/checkpoint_lstm.h5')
lstm.predict(train)

In [ ]:
for element in train:
    print(element)

In [ ]:
import tensorflow as tf
import numpy as np
from read_data import make_dataset

odoms,scans,labels=make_dataset("/media/louis/TOSHIBA EXT/FullData")


[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/43/auto_sending_goals_2021-06-30-10-06-18 already exists. Not creating.
[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/1/auto_sending_goals_2021-06-29-10-22-41 already exists. Not creating.
[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/10/auto_sending_goals_2021-06-29-10-30-24 already exists. Not creating.
[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/11/auto_sending_goals_2021-06-29-10-30-48 already exists. Not creating.
[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/12/auto_sending_goals_2021-06-29-10-33-03 already exists. Not creating.
[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/13/auto_sending_goals_2021-06-29-10-35-19 already exists. Not creating.
[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/14/auto_sending_goals_2021-06-29-10-37-34 already exists. Not creating.
[INFO]  Data folder /media/louis/TOSHIBA EXT/FullData/NoNoise/1

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
tf.data.Dataset.from_tensor_slices(odoms)

list

In [ ]:
list_data=[np.random.random((np.random.randint(200,500),13)).tolist() for i in range(10)]
test=tf.ragged.constant(list_data,ragged_rank=1)
ds=tf.data.Dataset.from_tensor_slices(test)

plop
(303, 13)
plop
(417, 13)
plop
(481, 13)
plop
(448, 13)
plop
(364, 13)
plop
(331, 13)
plop
(222, 13)
plop
(454, 13)
plop
(448, 13)
plop
(378, 13)
